## 구급 토큰화, 어간 추출, 표제어 추출
* 각 단어를 그 단어의 어간(stem)으로 표현해서 같은 어간을 가진 모든 단어를 구분해야 한다.
* 어간 추출(stemming): 일일이 어미를 찾아 제외하는 규칙 기반 방식
* 표제어 추출(lemmatization): 알려진 단어의 형태 사전(명시적이고 사람이 구축한 시스템)을 사용하고 문장에서 단어의 역할을 고려하는 처리 방식.
    * 표제어: 단어의 표준 형태
* 표제어 추출과 어간 추출은 단어의 단어의 일반 형태를 추출하는 정규화(normalization)의 한 형태로 볼 수 있다.

In [3]:
import spacy
import nltk

en_nlp = spacy.load('en_core_web_sm')
stemmer = nltk.stem.PorterStemmer()

def compare_normalization(doc):
    doc_spacy = en_nlp(doc)
    print('lemmatization:')
    print([token.lemma_ for token in doc_spacy])
    print('stem:')
    print([stemmer.stem(token.norm_.lower()) for token in doc_spacy])

In [4]:
compare_normalization(u"Our meeting today was worse than yesterday, I'm scared of meeting the clients tomorrow.")

lemmatization:
['our', 'meeting', 'today', 'be', 'bad', 'than', 'yesterday', ',', 'I', 'be', 'scared', 'of', 'meet', 'the', 'client', 'tomorrow', '.']
stem:
['our', 'meet', 'today', 'wa', 'wors', 'than', 'yesterday', ',', 'i', 'am', 'scare', 'of', 'meet', 'the', 'client', 'tomorrow', '.']


* 일반적으로 표제어 추출은 어간 추출보다 훨씬 복잡한 처리를 거친다.
* 머신러닝을 위해 토큰 정규화를 할 때는 어간 추출보다 좋은 결과를 낸다.

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

en_nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

def custom_tokenizer(document):
    doc_spacy = en_nlp(document)
    return [token.lemma_ for token in doc_spacy]

lemma_vect = CountVectorizer(tokenizer=custom_tokenizer, min_df=5)

In [6]:
from sklearn.datasets import load_files

reviews_train = load_files('data/aclImdb/train/')
text_train, y_train = reviews_train.data, reviews_train.target

X_train_lemma = lemma_vect.fit_transform(text_train)
print(f'X_train_lemma.shape:', X_train_lemma.shape)

vect = CountVectorizer(min_df=5).fit(text_train)
X_train = vect.transform(text_train)
print('X_train.shape:', X_train.shape)

X_train_lemma.shape: (25000, 23874)
X_train.shape: (25000, 27272)


In [10]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10]}
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.99, train_size=0.01, random_state=0)
grid = GridSearchCV(LogisticRegression(max_iter=5000), param_grid, cv=cv, n_jobs=-1)

grid.fit(X_train, y_train)
print(f'best score (CountVectorizer): {grid.best_score_:.3f}')

grid.fit(X_train_lemma, y_train)
print('best score (lemma): {grid.best_score_:.3f}')

best score (CountVectorizer): 0.717
best score (lemma): {grid.best_score_:.3f}
